In [1]:
import pandas as pd
import pyodbc
import numpy as np
import smtplib
from datetime import datetime, timedelta
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [2]:
lista_email_destinatarios = ['email_destinatario@gmail.com']
lista_email_suporte = ['email_suporte@gmail.com']

In [3]:
def divisao_com_zero(numerador, denominador):
    '''
    Faz a conta da divisão de dois números, caso o numerador ou o denominador for zero, ele retorna zero, para não dar erro.
    
    numerador(float): o numerador
    denominador(float): o denominador
    '''
    try:
        return np.where((denominador == 0) | (numerador == 0), 0, numerador / denominador)
    except ZeroDivisionError:
        return 0


def cor_indicador_acumulado(alcance_acumulado):
    
    '''
    Faz a formatação da cor do ícone em html do email entre verde (acima de 1), amarelo (acimad de 0,97) e vermelho.
    
    alcance_acumulado(float): número decimal representado a porcentagem do atingimeto da meta acumulada do mês até o dia anterior.
    
    '''
    
    
    if alcance_acumulado >= 1:
        cor_alcance_acumulado = 'green'
        return cor_alcance_acumulado
    elif alcance_acumulado > 0.97:
        cor_alcance_acumulado = 'yellow'
        return cor_alcance_acumulado
    else:
        cor_alcance_acumulado = 'red' 
        return cor_alcance_acumulado      
    
    
    
    
    
def cor_indicador_dia(alcance_dia):
    
    '''
    Faz a formatação da cor do ícone em html do email entre verde (acima de 1), amarelo (acimad de 0,97) e vermelho.
    
    alcance_dia(float): número decimal representado a porcentagem do atingimeto da meta do dia.
    
    '''
    
    
    if alcance_dia >= 1:
        cor_alcance_dia = 'green'
        return cor_alcance_dia
    elif alcance_dia > 0.97:
        cor_alcance_dia = 'yellow'
        return cor_alcance_dia
    else:
        cor_alcance_dia = 'red'
        return cor_alcance_dia
   

    
    
    
    
# Função para formatar a moeda no formato brasileiro "x.xxx,xx"
def formatar_numero(valor,tipo = 'moeda'):
    
    '''
    Faz a formatação dos números em moeda R$ substituindo o ponto pela vírgula e colocando o separador de milhar. No caso do
    número ser em porcentagem, substituir o ponto pela vírgula.
    
    valor(float): o número que vai ser transformado
    tipo: como padrão vai ser corversão em moeda, caso seja para porcentagem colocar "porcentagem"
    '''
    
    
    if tipo == 'porcentagem':
        valor = f'{valor:.2%}'
        valor = valor.replace('.',',')
        return valor
    elif tipo == 'numero':
        valor = f'{valor:,.2f}'
        valor = valor.replace(',','_').replace('.',',').replace('_','.')
        return valor
    else:
        valor = f'R$ {valor:,.0f}'
        valor = valor.replace(',','_').replace('.',',').replace('_','.')
        return valor
    



def enviar_email(lista_email, mensagem, assunto):
    
    
    '''
    Manda email de forma automática, utilizando a conta noreply@venancio.com.br para uma lista de email definida.
    
    
    '''
    
    # Configuração
    host = 'HOST'
    port = 'PORTA'
    user = 'EMAIL'
    password = 'SENHA'

    # Criando objeto
    server = smtplib.SMTP(host, port)

    # Login com servidor
    server.ehlo()
    server.starttls()
    server.login(user, password)

    # Criando mensagem
    message = mensagem
    email_msg = MIMEMultipart()
    email_msg['From'] = user
    
    email_msg['To'] = ', '.join(lista_email)
    email_msg['Subject'] = assunto
    email_msg.attach(MIMEText(message, 'html'))
    

    # Enviando mensagem
    server.sendmail(email_msg['From'], lista_email, email_msg.as_string())
    
    server.quit()



def converter_segundos(seconds):

    '''
    Faz a formatação dos segundos para o padrão %h%h:%min%min:%s%s
        
    seconds(int): o número que vai ser transformado

    '''

    if seconds < 0:
        seconds =abs(seconds)

        seconds = seconds % (24 * 3600) 
        hour = seconds // 3600
        seconds %= 3600
        minutes = seconds // 60
        seconds %= 60
        
        return "-%d:%02d:%02d" % (hour, minutes, seconds) 
    
    else:
        seconds = seconds % (24 * 3600) 
        hour = seconds // 3600
        seconds %= 3600
        minutes = seconds // 60
        seconds %= 60
        return "%d:%02d:%02d" % (hour, minutes, seconds)


In [4]:
## Query da Coleta de Dados


query_venda = pd.read_excel('vendas.xlsx', sheet_name='Vendas')




qurey_nao_presencial = pd.read_excel('vendas.xlsx', sheet_name='Não Presencial')



query_meta_nova = pd.read_excel('vendas.xlsx', sheet_name='Meta_Venda')



query_omni_colaborador = pd.read_excel('vendas.xlsx', sheet_name='COLABORADOR')


query_fidelidade_colaborador2 = pd.read_excel('vendas.xlsx', sheet_name='COLABORADOR2')


query_nps = pd.read_excel('vendas.xlsx', sheet_name='Satisfacao')


query_aniversario = pd.read_excel('vendas.xlsx', sheet_name='Aniversario')



quey_info_loja = pd.read_excel('vendas.xlsx', sheet_name='Info_Lojas')


query_dias_venda = pd.read_excel('vendas.xlsx', sheet_name='Dias_Vendas')



query_tma_colaborador2 = pd.read_excel('vendas.xlsx', sheet_name='TMA')



query_desempenho_colaboradors = pd.read_excel('vendas.xlsx', sheet_name='Desempenho')


query_tipo1 = pd.read_excel('vendas.xlsx', sheet_name='Industria')


query_campanha = pd.read_excel('vendas.xlsx', sheet_name='Campanhas')

In [5]:
#Pegando o dia de ontem para condição se não houver venda
data_ontem = datetime.now() -timedelta(days=1)

data_ontem_email = f'{data_ontem.day}/{data_ontem.month}'
data_ontem = data_ontem.strftime('%Y-%m-%d')

data_hoje = datetime.now()

data_hoje = f'{data_hoje.day}/{data_hoje.month}'

In [6]:
#Criando uma cópia do Dataframe para manipular
df_venda_omini = query_venda.copy()
df_venda_omini = df_venda_omini.sort_values(by=['LOJA','DATA'])
df_venda_omini['META'] = df_venda_omini['META'].fillna(df_venda_omini['VENDA'])
df_venda_omini.head(10)

,DATA,LOJA,VENDA,META,QTD_TOTAL,QTD_VENDA,VENDA_TOTAL
1966,2023-12-01,1,24739,24602,703,207,25541
384,2023-12-02,1,16436,17026,520,145,16461
724,2023-12-03,1,7483,6450,232,66,7472
368,2023-12-04,1,19433,23141,599,177,19552
1800,2023-12-05,1,20864,22300,659,197,20979
1018,2023-12-06,1,23505,24549,668,192,23426
875,2023-12-07,1,22082,20764,607,187,21988
976,2023-12-08,1,19455,24602,629,187,19226
1211,2023-12-09,1,12994,16544,452,135,12896
1446,2023-12-10,1,5435,6387,193,60,5435


In [7]:
#Pegando somente as lojas que estavam abertas no dia anterior.
ultima_data = df_venda_omini['DATA'].max()

lista_lojas = df_venda_omini.loc[df_venda_omini['DATA'] == data_ontem]

lista_lojas = list(set(lista_lojas['LOJA'].tolist()))

lista_lojas.sort()

In [8]:
query_meta_nova = query_meta_nova.sort_values(by=['LOJA','DATA'])
query_meta_nova.head()

,DATA,LOJA,VENDA,META,DESVIO,DIAS
281,1 - DESVIO_ATE_MOMENTO,1,369701,390845,-21144,9.8
349,2 - HOJE,1,0,23899,0,9.8
171,3 - FUTURO,1,0,188556,0,9.8
55,1 - DESVIO_ATE_MOMENTO,2,169253,200626,-31078,9.8
287,2 - HOJE,2,0,13874,0,9.8


In [9]:
#Criando uma cópia do Dataframe para manipular
df_meta_nova = query_meta_nova.copy()
df_meta_nova = df_meta_nova.sort_values(by=['LOJA','DATA'])
df_meta_nova = df_meta_nova.reset_index(drop=True)

nome_colunas = []


df_auxiliar = pd.DataFrame(
                        {'LOJA':[],
                        'META':[],
                        'DESVIO':[],
                        'META_SUGERIDA':[]
                        }
                        ) 


for loja in lista_lojas:

    linha = df_meta_nova['LOJA'] == loja
    coluna = df_meta_nova.columns.tolist()
    tabela_loja = df_meta_nova.loc[linha,coluna].reset_index(drop=True)

    print(lista_lojas)

    break


    if loja not in list(set(df_meta_nova['LOJA'].tolist())):

        pass


    else:
        if tabela_loja['DESVIO'][0] < 0 and df_meta_nova['DATA'][0] == '1 - DESVIO_ATE_MOMENTO':
            
            tabela_loja['DESVIO'] = abs(tabela_loja['DESVIO'])
            tabela_loja['DESVIO'] = tabela_loja['DESVIO'].fillna(tabela_loja['DESVIO'].max())


            tabela_loja['META_SUGERIDA'] = divisao_com_zero(tabela_loja['DESVIO'],tabela_loja['DIAS']) + tabela_loja['META']


        

            tabela_loja = tabela_loja[tabela_loja['DATA'] =='2 - HOJE']
            tabela_loja = tabela_loja[['LOJA','META','DESVIO','META_SUGERIDA']]

        else:

            tabela_loja = tabela_loja[tabela_loja['DATA'] =='2 - HOJE']

            tabela_loja['META_SUGERIDA'] = tabela_loja['META']

            tabela_loja = tabela_loja[['LOJA','META','DESVIO','META_SUGERIDA']]

            tabela_loja['DESVIO'] = tabela_loja['DESVIO'].fillna(0)


        df_auxiliar = pd.concat([df_auxiliar,tabela_loja])

df_meta_sugerida = df_auxiliar
df_meta_sugerida = df_meta_sugerida.rename(columns={'META':'META_DIA', 'DESVIO':'GAP_ANTERIOR'})
df_meta_sugerida['LOJA'] = df_meta_sugerida['LOJA'].astype('Int32')

df_meta_sugerida.head()
    

,LOJA,META_DIA,GAP_ANTERIOR,META_SUGERIDA


In [10]:
#Criando uma cópia do Dataframe para manipular
df_nao_presencial = qurey_nao_presencial.copy()
df_nao_presencial = df_nao_presencial.sort_values(by=['LOJA','DATA'])

df_nao_presencial.head(10)

,DATA,LOJA,CANAL_VENDA,VENDA_CANAL
35,2023-12-01,1,FSDKH,418
595,2023-12-01,1,FBIJD,43
2822,2023-12-01,1,VMDIC,19176
8296,2023-12-01,1,EDYJC,983
11503,2023-12-01,1,FNIUL,274
12290,2023-12-01,1,QXILV,385
2605,2023-12-02,1,ZAZDU,60
3990,2023-12-02,1,VMDIC,12083
4748,2023-12-02,1,EDYJC,1827
5301,2023-12-02,1,FSDKH,10


In [11]:
#Criando uma cópia do Dataframe para manipular
df_omni_colaborador = query_omni_colaborador.copy()
df_omni_colaborador = df_omni_colaborador.sort_values(by=['LOJA','DATA'])
df_omni_colaborador['TOTAL_QTD'] = df_omni_colaborador['QTD_ORIGINADOS'] + df_omni_colaborador['QTD_FATURADOS']

df_omni_colaborador = df_omni_colaborador.groupby(['LOJA','COLABORADOR']).sum(['QTD_ORIGINADOS','VENDA_ORIGEM','QTD_FATURADOS','VENDA_FATURADO','TOTAL_QTD'])
df_omni_colaborador = df_omni_colaborador.reset_index(drop=False)


df_omni_colaborador.tail(10)

,LOJA,COLABORADOR,QTD_ORIGINADOS,VENDA_ORIGEM,QTD_FATURADOS,VENDA_FATURADO,TOTAL_QTD
1085,555,ACJSD,30,109.100303,6,3.726364,36
1086,555,KJXPA,0,0.000000,12,0.476364,12
1087,555,OYHAX,0,0.000000,6,10.538182,6
1088,555,QIPGR,24,154.241818,0,0.000000,24
1089,555,QMGWZ,78,252.195152,12,26.033636,90
1090,555,SBFTP,60,388.559394,42,97.724545,102
1091,555,SIVIE,12,91.167576,0,0.000000,12
1092,555,TUBPQ,6,19.252121,0,0.000000,6
1093,555,XHUCR,36,397.311515,36,70.758182,72
1094,555,XZKUM,108,517.164242,72,211.420000,180


In [12]:
#Criando uma cópia do Dataframe para manipular
df_fidelidade_colaborador2 = query_fidelidade_colaborador2.copy()
df_fidelidade_colaborador2 = df_fidelidade_colaborador2.sort_values(by=['LOJA','DATA'])
df_fidelidade_colaborador2['% CONVERSÃO'] = divisao_com_zero(df_fidelidade_colaborador2['QTD_VENDA_FIDELIDADE'],df_fidelidade_colaborador2['QTD_VENDA_TOTAL'])
df_fidelidade_colaborador2['% META'] = 0.9
df_fidelidade_colaborador2['% ALCANCE'] = divisao_com_zero(df_fidelidade_colaborador2['% CONVERSÃO'],df_fidelidade_colaborador2['% META'])

df_fidelidade_colaborador2.head(10)

,DATA,LOJA,COLABORADOR,QTD_VENDA_FIDELIDADE,QTD_VENDA_TOTAL,% CONVERSÃO,% META,% ALCANCE
0,2023-12-01,1,DFHVQ,51,53,0.962264,0.9,1.069182
1,2023-12-01,1,UCAWS,92,97,0.948454,0.9,1.053837
2,2023-12-01,1,FKEBC,29,30,0.966667,0.9,1.074074
3,2023-12-01,1,SBHJU,66,84,0.785714,0.9,0.873016
4,2023-12-01,1,BGJNC,60,69,0.869565,0.9,0.966184
5,2023-12-01,1,XQZSQ,110,116,0.948276,0.9,1.053640
6,2023-12-01,1,DIHTQ,0,-6,0.000000,0.9,0.000000
7,2023-12-01,1,XDFXX,70,82,0.853659,0.9,0.948509
8,2023-12-01,1,MEWAF,64,70,0.914286,0.9,1.015873
9,2023-12-02,1,XQZSQ,62,67,0.925373,0.9,1.028192


In [13]:
#Criando uma cópia do Dataframe para manipular
df_nps= query_nps.copy()
df_nps = df_nps.sort_values(by=['LOJA','DATA'])

df_nps = df_nps.groupby(['LOJA']).sum(['TOTAL_RESPOSTAS','PROMOTORES','DETRATORES'])
df_nps = df_nps.reset_index(drop=False)
df_nps['NPS - NOTA'] = (df_nps['PROMOTORES'] / df_nps['TOTAL_RESPOSTAS']) - (df_nps['DETRATORES'] / df_nps['TOTAL_RESPOSTAS'])
df_nps['NPS - NOTA'] = df_nps['NPS - NOTA'].apply(lambda x: formatar_numero(x, 'porcentagem'))
df_nps['% CONVERSÃO'] = df_nps['TOTAL_RESPOSTAS'] / df_nps['QTD']
df_nps['% CONVERSÃO'] = df_nps['% CONVERSÃO'].apply(lambda x: formatar_numero(x, 'porcentagem'))
df_nps['TOTAL_RESPOSTAS'] = df_nps['TOTAL_RESPOSTAS'].apply(lambda x: formatar_numero(x, 'numero'))
df_nps = df_nps.drop(['PROMOTORES','DETRATORES','QTD'], axis=1)

df_nps.tail(10)

,LOJA,TOTAL_RESPOSTAS,NPS - NOTA,% CONVERSÃO
68,117,"2.071,00","93,82%","67,39%"
69,119,"1.909,00","94,39%","53,03%"
70,121,"1.056,00","93,18%","18,98%"
71,123,"2.455,00","93,48%","39,24%"
72,124,"3.228,00","90,89%","83,98%"
73,125,"1.567,00","87,75%","52,36%"
74,126,"794,00","91,44%","41,03%"
75,131,"4.482,00","91,90%","75,65%"
76,132,"4.159,00","96,85%","47,83%"
77,555,"1.406,00","88,69%","22,11%"


In [14]:
#Criando uma cópia do Dataframe para manipular
df_aniversario = query_aniversario.copy()
df_aniversario = df_aniversario.sort_values(by=['LOJA'])

df_aniversario.head(10)

,ID,LOJA,NOME
0,XXXXX,5,CARLA
1,YYYYY,6,ANDRE
2,ZZZZZ,21,MARCOS
3,AAAA,27,ALINE
4,BBBBB,34,SARAH
5,CCCCC,44,OTAVIO
6,DDDDD,89,ALESSANDRA
7,EEEEEE,114,TALES


In [15]:
numero_dias = query_dias_venda.iloc[0,0]

#Criando uma cópia do Dataframe para manipular
df_info_loja = quey_info_loja.copy()
df_info_loja = df_info_loja.sort_values(by=['LOJA'])

df_info_loja.tail(10)

,LOJA,NM_LOJA,NM_SUPERVISOR_LOJA,EMAIL_SUPERVISOR,EMAIL_GERENTE,EMAIL_COLABORADOR
110,119,119-NZWHD,VMFWN,VMFWN@email.com,gerente_119@email.com,colaborador_119@email.com
40,121,121-YACRY,BPRIJ,BPRIJ@email.com,gerente_121@email.com,colaborador_121@email.com
21,123,123-KKWYN,WKOIZ,WKOIZ@email.com,gerente_123@email.com,colaborador_123@email.com
22,124,124-HADTF,WKOIZ,WKOIZ@email.com,gerente_124@email.com,colaborador_124@email.com
108,125,125-HUHRC,IPOXZ,IPOXZ@email.com,gerente_125@email.com,colaborador_125@email.com
109,126,126-AYLZN,VMFWN,VMFWN@email.com,gerente_126@email.com,colaborador_126@email.com
76,131,131-ETUJS,SFPSB,SFPSB@email.com,gerente_131@email.com,colaborador_131@email.com
77,132,132-OHDFQ,SFPSB,SFPSB@email.com,gerente_132@email.com,colaborador_132@email.com
23,515,515-ENZCH,WKOIZ,WKOIZ@email.com,gerente_15@email.com,colaborador_15@email.com
75,555,555-SICTH,SFPSB,SFPSB@email.com,gerente_55@email.com,colaborador_55@email.com


In [16]:
#Criando uma cópia do Dataframe para manipular
df_tma_colaborador2 = query_tma_colaborador2.copy()

df_tma_colaborador2 = df_tma_colaborador2.sort_values(by=['LOJA'])

df_tma_colaborador2 = df_tma_colaborador2.reset_index(drop=True)


df_tma_colaborador2.head(5)

,LOJA,COLABORADOR,TEMPO_MEDIO,MEDIANA_LOJA,DIFF_MEDIANA_COLABORADOR,MEDIANA_EMPRESA,DIFF_MEDIANA_EMPRESA,QTD_VENDA,OPERACOES,ITENS_QTD_VENDA,MEDIANA_ITENS_QTD_VENDA_LOJA
0,1,YFEOJ,60,67,-8,74,-15,5235,1586,3,3
1,1,LYTSH,77,67,10,74,3,3305,1052,3,3
2,1,EHVLD,65,67,-3,74,-10,4480,1412,3,3
3,1,DCIFD,144,67,77,74,70,616,201,3,3
4,1,GHVNY,59,67,-9,74,-16,6057,1808,3,3


In [17]:
#Criando uma cópia do Dataframe para manipular
df_desempenho_colaboradors = query_desempenho_colaboradors.copy()


df_desempenho_colaboradors = df_desempenho_colaboradors.sort_values(by=['LOJA'])

df_desempenho_colaboradors = df_desempenho_colaboradors.reset_index(drop=True)

df_desempenho_colaboradors['DIFF_MEDIANA_VENDA'] = df_desempenho_colaboradors['VENDA'] - df_desempenho_colaboradors['MEDIANA_VENDA_LOJA']

df_desempenho_colaboradors['DIFF_MEDIANA_ITEM_CUPOM'] = df_desempenho_colaboradors['ITENS_VENDA'] - df_desempenho_colaboradors['MEDIANA_ITENS_VENDA_LOJA']

df_desempenho_colaboradors['ITENS_VENDA_COM_TIPO1'] = divisao_com_zero(df_desempenho_colaboradors['QTD_TIPO1'], df_desempenho_colaboradors['QTD_VENDA'])

df_desempenho_colaboradors['DIFF_MEDIANA_ITENS_VENDA_COM_TIPO1'] = df_desempenho_colaboradors['ITENS_VENDA_COM_TIPO1'] -  df_desempenho_colaboradors['MEDIANA_PERC_QTD_TIPO1_LOJA']

df_desempenho_colaboradors = df_desempenho_colaboradors[['LOJA', 'COLABORADOR','VENDA','MEDIANA_VENDA_LOJA','DIFF_MEDIANA_VENDA',
                                                       'ITENS_VENDA','MEDIANA_ITENS_VENDA_LOJA','DIFF_MEDIANA_ITEM_CUPOM','ITENS_VENDA_COM_TIPO1',
                                                       'MEDIANA_PERC_QTD_TIPO1_LOJA','DIFF_MEDIANA_ITENS_VENDA_COM_TIPO1']]



df_desempenho_colaboradors.head()


,LOJA,COLABORADOR,VENDA,MEDIANA_VENDA_LOJA,DIFF_MEDIANA_VENDA,ITENS_VENDA,MEDIANA_ITENS_VENDA_LOJA,DIFF_MEDIANA_ITEM_CUPOM,ITENS_VENDA_COM_TIPO1,MEDIANA_PERC_QTD_TIPO1_LOJA,DIFF_MEDIANA_ITENS_VENDA_COM_TIPO1
0,1,FYBHC,51818,38806,13012,1,1,0,0.185279,1,-0.814721
1,1,FDKTX,52159,38806,13353,1,1,0,0.104956,1,-0.895044
2,1,YRSQO,38806,38806,0,1,1,0,0.188088,1,-0.811912
3,1,RWXRR,35133,38806,-3673,1,1,0,0.152941,1,-0.847059
4,1,NAGPF,47809,38806,9003,1,1,0,0.121655,1,-0.878345


In [18]:
#Criando uma cópia do Dataframe para manipular
df_tipo1 = query_tipo1.copy()


df_tipo1.head(10)

,DATA,LOJA,COLABORADOR,QTD_PRODUTOS_TIPO1,QTD_PRODUTOS_TIPO1_VENDIDOS_DESCONTO,VL_VENDA
0,2023-12-01,1,QFPUT,5,5,4899
1,2023-12-01,1,RNAZN,12,11,7528
2,2023-12-01,1,FAFZX,3,3,2308
3,2023-12-01,1,QFPUT,8,7,5265
4,2023-12-01,1,RNAZN,13,13,8108
5,2023-12-01,1,FAFZX,6,3,20614
6,2023-12-01,1,WJIJM,4,3,3514
7,2023-12-02,1,MFBBO,1,1,1341
8,2023-12-02,1,DQFQO,4,3,3008
9,2023-12-02,1,EKAGA,6,6,4754


In [19]:
#Criando uma cópia do Dataframe para manipular
df_campanha = query_campanha.copy()
df_campanha = df_campanha.sort_values(by=['LOJA'])

df_campanha['DESVIO'] = df_campanha['QTD_VENDA'] - df_campanha['QTD_META']
df_campanha['ALCANCE'] = divisao_com_zero(df_campanha['QTD_VENDA'], df_campanha['QTD_META'])
df_campanha['RANKING_CLUSTER'] = df_campanha.groupby(['CAMPANHA','CLUSTER'])['ALCANCE'].rank(ascending=False,method='min')

df_campanha['PARTCIPACAO_CAMPANHA_LOJA'] = divisao_com_zero(df_campanha['VENDA_CAMPANHA'], df_campanha['VENDA'])

df_campanha['PARTCIPACAO_CAMPANHA_LOJA'] = df_campanha['PARTCIPACAO_CAMPANHA_LOJA'].apply(lambda x: formatar_numero(x, 'porcentagem'))

df_campanha['ALCANCE'] = df_campanha['ALCANCE'].apply(lambda x: formatar_numero(x, 'porcentagem'))
df_campanha['QTD_TOTAL'] = df_campanha['QTD_VENDA'].apply(lambda x: formatar_numero(x, 'numero'))
df_campanha['QTD_META'] = df_campanha['QTD_META'].apply(lambda x: formatar_numero(x, 'numero'))
df_campanha['TICKET_MEDIO'] = df_campanha['TICKET_MEDIO'].apply(lambda x: formatar_numero(x, 'moeda'))
df_campanha['VENDA_CAMPANHA'] = df_campanha['VENDA_CAMPANHA'].apply(lambda x: formatar_numero(x, 'moeda'))

df_campanha[['DATA_INICIAL', 'DATA_FINAL']] = df_campanha[['DATA_INICIAL', 'DATA_FINAL']].apply(lambda x: pd.to_datetime(x).dt.strftime('%d/%m/%Y'))


df_campanha.head(10)

,LOJA,CAMPANHA,CLUSTER,PUBLICO,DATA_INICIAL,DATA_FINAL,QTD_VENDA,QTD_META,VENDA_CAMPANHA,TICKET_MEDIO,VENDA,VENDA_TOTAL,DESVIO,ALCANCE,RANKING_CLUSTER,PARTCIPACAO_CAMPANHA_LOJA,QTD_TOTAL
0,1,LXGII,CLUSTER 2,PUBLICO 1,01/12/2023,31/01/2024,11,"30,00",R$ 620,R$ 29,739401,740769,-19,"36,67%",4.0,"0,08%","11,00"
22,1,EGIYG,CLUSTER 3,PUBLICO 3,01/12/2023,31/01/2024,15,"50,00",R$ 1.219,R$ 47,739401,740769,-35,"30,00%",4.0,"0,16%","15,00"
21,1,LCBCZ,CLUSTER 2,PUBLICO 1,01/12/2023,31/01/2024,24,"50,00",R$ 822,R$ 20,739401,740769,-26,"48,00%",18.0,"0,11%","24,00"
20,1,WKUGC,CLUSTER 2,PUBLICO 4,01/12/2023,31/12/2023,103,"270,00",R$ 4.802,R$ 27,739401,740769,-167,"38,15%",18.0,"0,65%","103,00"
19,1,IGLGR,CLUSTER 2,PUBLICO 1,01/12/2023,31/01/2024,35,"90,00",R$ 1.713,R$ 28,739401,740769,-55,"38,89%",22.0,"0,23%","35,00"
18,1,PPUBU,CLUSTER 2,PUBLICO 5,01/12/2023,15/12/2023,188,"334,00",R$ 1.333,R$ 8,602847,603723,-146,"56,29%",13.0,"0,22%","188,00"
17,1,ZEOWZ,CLUSTER 3,PUBLICO 3,01/12/2023,31/01/2024,1,"10,00",R$ 55,R$ 55,739401,740769,-9,"10,00%",17.0,"0,01%","1,00"
15,1,ZZMLF,CLUSTER 3,PUBLICO 3,01/12/2023,31/01/2024,5,"20,00",R$ 449,R$ 45,739401,740769,-15,"25,00%",25.0,"0,06%","5,00"
14,1,IPNBZ,CLUSTER 2,PUBLICO 5,01/12/2023,31/12/2023,30,"50,00",R$ 1.701,R$ 37,739401,740769,-20,"60,00%",8.0,"0,23%","30,00"
13,1,WCGBP,CLUSTER 2,PUBLICO 2,01/12/2023,31/12/2023,21,"45,00",R$ 2.161,R$ 59,739401,740769,-24,"46,67%",14.0,"0,29%","21,00"


In [25]:
#Listar todas as lojas do dataframe de Vendas Omni

nome_colunas = []

#Percorrer essas lojas e criar uma sub- base para cada loja
for loja in lista_lojas:


    #INDICADORES DE VENDA OMNI
    
    linha = df_venda_omini['LOJA'] == loja
    coluna = df_venda_omini.columns.tolist()
    tabela_loja = df_venda_omini.loc[linha,coluna].reset_index(drop=True)


    ultima_data = tabela_loja['DATA'].max()


    #INFORMAÇÕES DA LOJA

    linha = df_info_loja['LOJA'] == loja
    coluna = df_info_loja.columns.tolist()
    tabela_info_loja = df_info_loja.loc[linha,coluna].reset_index(drop=True)

    lista_email_loja = []

    nome_loja = tabela_info_loja['NM_LOJA'][0]
    id_loja = tabela_info_loja['LOJA'][0]
    email_gerente = tabela_info_loja['EMAIL_GERENTE'][0]
    email_colaborador = tabela_info_loja['EMAIL_COLABORADOR'][0]
    email_supervisor = tabela_info_loja['EMAIL_SUPERVISOR'][0]

    lista_email_loja.append(email_gerente)
    lista_email_loja.append(email_colaborador)
    lista_email_loja.append(email_supervisor)

    if (ultima_data != data_ontem) | loja > 555 :

        pass

    else:         

        #Pegar os indicadores de venda da loja
        base_dia = tabela_loja.loc[tabela_loja['DATA'] == ultima_data]


        venda_ultimo_dia = sum(base_dia['VENDA'])
        meta_ultimo_dia = sum(base_dia['META'])
        desvio_venda_ultimo_dia = venda_ultimo_dia - meta_ultimo_dia
        alcance_venda_ultimo_dia = divisao_com_zero(venda_ultimo_dia , meta_ultimo_dia)

        venda_acumulado = sum(tabela_loja['VENDA'])
        meta_acumulada = sum(tabela_loja['META'])
        desvio_acumulado = venda_acumulado - meta_acumulada
        alcance_acumulado = divisao_com_zero(venda_acumulado , meta_acumulada)


        #Calcular o Volume por Cupom do mês
        volume_cupom_acumulado = divisao_com_zero(sum(tabela_loja['QTD_TOTAL']), sum(tabela_loja['QTD_VENDA']))


    
        #INDICADORES DE NÃO PRESENCIAL NO MÊS

        df_loja_nao_presencial = df_nao_presencial[df_nao_presencial['LOJA'] == loja]

        colunas_desejadas = ['CANAL_VENDA','VENDA_CANAL']

        tabela_loja_nao_presencial = df_loja_nao_presencial[colunas_desejadas].reset_index(drop=True)

        tabela_loja_nao_presencial = tabela_loja_nao_presencial.groupby(['CANAL_VENDA']).sum()

        tabela_loja_nao_presencial = tabela_loja_nao_presencial.reset_index(drop=False)

        tabela_loja_nao_presencial = tabela_loja_nao_presencial.sort_values(by='VENDA_CANAL', ascending=False )        

        nao_presencial_total = sum(tabela_loja_nao_presencial['VENDA_CANAL'])

        tabela_loja_nao_presencial['% TOTAL']  =  tabela_loja_nao_presencial['VENDA_CANAL'] / nao_presencial_total

        tabela_loja_nao_presencial['VENDA_CANAL'] = tabela_loja_nao_presencial['VENDA_CANAL'].apply(lambda x: formatar_numero(x,'moeda'))

        tabela_loja_nao_presencial['% TOTAL'] = tabela_loja_nao_presencial['% TOTAL'].apply(lambda x: formatar_numero(x,'porcentagem'))



        
        
        #INDICADORES DA META SUGERIDA E TICKET MÉDIO
        

        df_loja_meta_sugerida = df_meta_sugerida[df_meta_sugerida['LOJA'] == loja]

        meta_dia = sum(df_loja_meta_sugerida['META_DIA'])
        gap_ontem = sum(df_loja_meta_sugerida['GAP_ANTERIOR'])
        meta_sugeriada = sum(df_loja_meta_sugerida['META_SUGERIDA'])
        clietes_acumulado = sum(tabela_loja['QTD_VENDA'])
        qtd_dias = numero_dias
        media_cliente_dia = divisao_com_zero(clietes_acumulado , qtd_dias)
        ticket_medio = divisao_com_zero(meta_sugeriada , media_cliente_dia)

        tabela_meta_email = pd.DataFrame(
                                        {'Indicador':['Meta do Dia','Gap no Mês','Meta Sugerida no Dia','Clientes do Mês','Dias','Clientes/Dia','Ticket Médio Sugerido'],
                                         'Valor':[meta_dia,gap_ontem,meta_sugeriada,clietes_acumulado,qtd_dias,media_cliente_dia,ticket_medio]
                                        }
                                        )
        
        condicao_numeros = tabela_meta_email['Indicador'].isin(['Clientes do Mês', 'Dias', 'Clientes/Dia'])
        tabela_meta_email.loc[condicao_numeros, 'Valor'] = tabela_meta_email.loc[condicao_numeros, 'Valor'].apply(lambda x: formatar_numero(x, 'numero'))

        condicao_moeda = tabela_meta_email['Indicador'].isin(['Meta do Dia', 'Gap no Mês', 'Meta Sugerida no Dia', 'Ticket Médio Sugerido'])
        tabela_meta_email.loc[condicao_moeda, 'Valor'] = tabela_meta_email.loc[condicao_moeda, 'Valor'].apply(lambda x: formatar_numero(x, 'moeda'))

        

        #INDICADORES DA VITRINE INFINITA POR COLABORADOR

        tabela_loja_omni_colaborador = df_omni_colaborador[df_omni_colaborador['LOJA'] == loja]

        colunas_desejadas = ['COLABORADOR','QTD_ORIGINADOS','VENDA_ORIGEM','QTD_FATURADOS','VENDA_FATURADO','TOTAL_QTD']

        tabela_loja_omni_colaborador = tabela_loja_omni_colaborador[colunas_desejadas].reset_index(drop=True)

        tabela_loja_omni_colaborador = tabela_loja_omni_colaborador.sort_values(['QTD_ORIGINADOS'], ascending=False)

        tabela_loja_omni_colaborador = tabela_loja_omni_colaborador.rename(columns={'COLABORADOR': 'FUNCIONARIO'})

        total_loja_omni_colaborador = pd.DataFrame(
                                                  {'LOJA': [nome_loja], 'QTD_ORIGINADOS': [sum(tabela_loja_omni_colaborador['QTD_ORIGINADOS'])] 	
                                                   ,'VENDA_ORIGEM': [sum(tabela_loja_omni_colaborador['VENDA_ORIGEM'])], 'QTD_FATURADOS': [sum(tabela_loja_omni_colaborador['QTD_FATURADOS'])],
                                                   'VENDA_FATURADO': [sum(tabela_loja_omni_colaborador['VENDA_FATURADO'])], 'TOTAL_QTD': [sum(tabela_loja_omni_colaborador['TOTAL_QTD'])]
                                                  }      
                                                  )
        

        tabela_loja_omni_colaborador['VENDA_ORIGEM'] = tabela_loja_omni_colaborador['VENDA_ORIGEM'].apply(lambda x: formatar_numero(x, 'moeda'))

        tabela_loja_omni_colaborador['VENDA_FATURADO'] = tabela_loja_omni_colaborador['VENDA_FATURADO'].apply(lambda x: formatar_numero(x, 'moeda'))

        total_loja_omni_colaborador['VENDA_ORIGEM'] = total_loja_omni_colaborador['VENDA_ORIGEM'].apply(lambda x: formatar_numero(x, 'moeda'))

        total_loja_omni_colaborador['VENDA_FATURADO'] = total_loja_omni_colaborador['VENDA_FATURADO'].apply(lambda x: formatar_numero(x, 'moeda'))




        #INDICADORES DE FIDELIDADE
    
        linha = df_fidelidade_colaborador2['LOJA'] == loja
        coluna = df_fidelidade_colaborador2.columns.tolist()
        tabela_loja_fidelidade = df_fidelidade_colaborador2.loc[linha,coluna].reset_index(drop=True)

        base_dia_fidelidade = tabela_loja_fidelidade[tabela_loja_fidelidade['DATA'] == ultima_data].reset_index(drop=True)
        base_dia_fidelidade = base_dia_fidelidade.drop(['DATA','LOJA'], axis=1)


        base_dia_fidelidade = base_dia_fidelidade.sort_values(['QTD_VENDA_FIDELIDADE'], ascending=False)

        meta_total = base_dia_fidelidade['% META'][0]
        conversao_total = divisao_com_zero(sum(base_dia_fidelidade['QTD_VENDA_FIDELIDADE']), sum(base_dia_fidelidade['QTD_VENDA_TOTAL']))
        alcance_total = divisao_com_zero(conversao_total,meta_total)


        total_loja_fidelidade = pd.DataFrame(
                                        {'LOJA': [nome_loja], 'QTD_VENDA_FIDELIDADE': [sum(base_dia_fidelidade['QTD_VENDA_FIDELIDADE'])] 	
                                        ,'QTD_VENDA_TOTAL': [sum(base_dia_fidelidade['QTD_VENDA_TOTAL'])], '% CONVERSÃO': [conversao_total],
                                        '% META': [meta_total], '% ALCANCE': [alcance_total]
                                        }      
                                        )
        

        base_dia_fidelidade['% CONVERSÃO'] = base_dia_fidelidade['% CONVERSÃO'].apply(lambda x: formatar_numero(x, 'porcentagem'))
        base_dia_fidelidade['% META'] = base_dia_fidelidade['% META'].apply(lambda x: formatar_numero(x, 'porcentagem'))
        base_dia_fidelidade['% ALCANCE'] = base_dia_fidelidade['% ALCANCE'].apply(lambda x: formatar_numero(x, 'porcentagem'))


        total_loja_fidelidade['% CONVERSÃO'] = total_loja_fidelidade['% CONVERSÃO'].apply(lambda x: formatar_numero(x, 'porcentagem'))
        total_loja_fidelidade['% META'] = total_loja_fidelidade['% META'].apply(lambda x: formatar_numero(x, 'porcentagem'))
        total_loja_fidelidade['% ALCANCE'] = total_loja_fidelidade['% ALCANCE'].apply(lambda x: formatar_numero(x, 'porcentagem'))


        

        #INDICADORES DE NPS NO MÊS
        

        df_loja_nps = df_nps[df_nps['LOJA'] == loja]

        if df_loja_nps.empty:
            tabela_nps = "Não tem Informação de Nota"

        else:
            tabela_nps = df_loja_nps.to_html(index=False)


        

        #INDICADORES DE ANIVERSARIO
    
        linha = df_aniversario['LOJA'] == loja
        tabela_loja_aniversario = df_aniversario.loc[linha].reset_index(drop=True)
        tabela_loja_aniversario = tabela_loja_aniversario.drop(['ID_CHAPA_LEGADO'], axis=1)

        if tabela_loja_aniversario.empty:
            emoji ="😞"
            tabela_aniversario = f"Não tem Aniversariante hoje {emoji}"

        else:
            tabela_aniversario = tabela_loja_aniversario.to_html(index=False)
        emoji = "🎂"

        


        #INDICADORES DE TMA
    
        linha = df_tma_colaborador2['LOJA'] == loja
        coluna = df_tma_colaborador2.columns.tolist()
        tabela_loja_tma = df_tma_colaborador2.loc[linha,coluna].reset_index(drop=True)
        tabela_loja_tma = tabela_loja_tma.sort_values(['TEMPO_MEDIO'], ascending=False)

        tabela_loja_tma[['TEMPO_MEDIO', 'MEDIANA_LOJA','DIFF_MEDIANA_COLABORADOR','MEDIANA_EMPRESA','DIFF_MEDIANA_EMPRESA']] = tabela_loja_tma[['TEMPO_MEDIO', 'MEDIANA_LOJA',
        'DIFF_MEDIANA_COLABORADOR','MEDIANA_EMPRESA','DIFF_MEDIANA_EMPRESA']].apply(lambda col: col.apply(converter_segundos))
  
        tabela_loja_tma['QTD_VENDA'] = tabela_loja_tma['QTD_VENDA'].apply(lambda x: formatar_numero(x, 'numero'))
        tabela_loja_tma['OPERACOES'] = tabela_loja_tma['OPERACOES'].apply(lambda x: formatar_numero(x, 'numero'))

        tabela_loja_tma = tabela_loja_tma.drop(['LOJA'], axis=1)

        tabela_loja_tma = tabela_loja_tma.drop(['MEDIANA_LOJA','MEDIANA_EMPRESA'], axis=1)


        tabela_loja_tma = tabela_loja_tma.rename(columns={'DIFF_MEDIANA_COLABORADOR': 'DIFF_MEDIANA_FUNC.',
                                                               'DIFF_MEDIANA_EMPRESA':'DIFF_MEDIANA_EMP.'})


   
        
        #INDICADORES DE DESEMPENHO DOS COLABORADOR
    
        linha = df_desempenho_colaboradors['LOJA'] == loja
        coluna = df_desempenho_colaboradors.columns.tolist()
        tabela_loja_desempenho_colaborador= df_desempenho_colaboradors.loc[linha,coluna].reset_index(drop=True)

        tabela_loja_desempenho_colaborador = tabela_loja_desempenho_colaborador.sort_values(['ITENS_VENDA'], ascending=True)

        tabela_loja_desempenho_colaborador[['VENDA','MEDIANA_VENDA_LOJA','DIFF_MEDIANA_VENDA']] = tabela_loja_desempenho_colaborador[['VENDA','MEDIANA_VENDA_LOJA','DIFF_MEDIANA_VENDA']].apply(lambda col: col.apply(formatar_numero))

  
        tabela_loja_desempenho_colaborador['ITENS_VENDA'] = tabela_loja_desempenho_colaborador['ITENS_VENDA'].apply(lambda x: formatar_numero(x, 'numero'))
        tabela_loja_desempenho_colaborador['MEDIANA_ITENS_VENDA_LOJA'] = tabela_loja_desempenho_colaborador['MEDIANA_ITENS_VENDA_LOJA'].apply(lambda x: formatar_numero(x, 'numero'))
        tabela_loja_desempenho_colaborador['DIFF_MEDIANA_ITEM_CUPOM'] = tabela_loja_desempenho_colaborador['DIFF_MEDIANA_ITEM_CUPOM'].apply(lambda x: formatar_numero(x, 'numero'))
        tabela_loja_desempenho_colaborador['QTD_TIPO1'] = tabela_loja_desempenho_colaborador['QTD_TIPO1'].apply(lambda x: formatar_numero(x, 'porcentagem'))
        tabela_loja_desempenho_colaborador['MEDIANA_PERC_QTD_TIPO1_LOJA'] = tabela_loja_desempenho_colaborador['MEDIANA_PERC_QTD_TIPO1_LOJA'].apply(lambda x: formatar_numero(x, 'porcentagem'))
        tabela_loja_desempenho_colaborador['DIFF_MEDIANA_ITENS_VENDA_COM_TIPO1'] = tabela_loja_desempenho_colaborador['DIFF_MEDIANA_ITENS_VENDA_COM_TIPO1'].apply(lambda x: formatar_numero(x, 'porcentagem'))
        
        tabela_loja_desempenho_colaborador = tabela_loja_desempenho_colaborador.drop(['LOJA'], axis=1)

        tabela_loja_desempenho_colaborador = tabela_loja_desempenho_colaborador.drop(['MEDIANA_ITENS_VENDA_LOJA','MEDIANA_VENDA_LOJA','MEDIANA_PERC_QTD_TIPO1_LOJA'], axis=1)


        

        #INDICADORES DE TIPO1
    
        linha = df_tipo1['LOJA'] == loja
        coluna = df_tipo1.columns.tolist()
        tabela_tipo1_loja = df_tipo1.loc[linha,coluna].reset_index(drop=True).copy()

        tabela_tipo1_dia = tabela_tipo1_loja.loc[tabela_tipo1_loja['DATA'] == ultima_data].copy()
        tabela_tipo1_dia['TAXA_CONVERSAO'] = divisao_com_zero(tabela_tipo1_dia['QTD_PRODUTOS_TIPO1_VENDIDOS_DESCONTO'], tabela_tipo1_dia['QTD_PRODUTOS_TIPO1']).copy()
        tabela_tipo1_dia = tabela_tipo1_dia[['COLABORADOR','TAXA_CONVERSAO','QTD_PRODUTOS_TIPO1_VENDIDOS_DESCONTO','QTD_PRODUTOS_TIPO1','VL_VENDA']].copy()
        tabela_tipo1_dia = tabela_tipo1_dia.sort_values(['TAXA_CONVERSAO'], ascending=False).copy()

        conversao_tipo1_total = divisao_com_zero(sum(tabela_tipo1_dia['QTD_PRODUTOS_TIPO1_VENDIDOS_DESCONTO']), sum(tabela_tipo1_dia['QTD_PRODUTOS_TIPO1']))

        

        total_loja_tipo1 = pd.DataFrame(
                                    {'LOJA': [nome_loja], 'TAXA_CONVERSAO': [conversao_tipo1_total], 
                                    'QTD_PRODUTOS_TIPO1_VENDIDOS_DESCONTO': [sum(tabela_tipo1_dia['QTD_PRODUTOS_TIPO1_VENDIDOS_DESCONTO'])], 
                                    'QTD_PRODUTOS_TIPO1': [sum(tabela_tipo1_dia['QTD_PRODUTOS_TIPO1'])],
                                    'VL_VENDA': [sum(tabela_tipo1_dia['VL_VENDA'])]
                                    }      
                                    )
        

        tabela_tipo1_dia['TAXA_CONVERSAO'] = tabela_tipo1_dia['TAXA_CONVERSAO'].apply(lambda x: formatar_numero(x, 'porcentagem'))
        tabela_tipo1_dia['QTD_PRODUTOS_TIPO1_VENDIDOS_DESCONTO'] = tabela_tipo1_dia['QTD_PRODUTOS_TIPO1_VENDIDOS_DESCONTO'].apply(lambda x: formatar_numero(x, 'numero'))
        tabela_tipo1_dia['QTD_PRODUTOS_TIPO1'] = tabela_tipo1_dia['QTD_PRODUTOS_TIPO1'].apply(lambda x: formatar_numero(x, 'numero'))
        tabela_tipo1_dia['VL_VENDA'] = tabela_tipo1_dia['VL_VENDA'].apply(lambda x: formatar_numero(x, 'moeda'))


        total_loja_tipo1['TAXA_CONVERSAO'] = total_loja_tipo1['TAXA_CONVERSAO'].apply(lambda x: formatar_numero(x, 'porcentagem'))
        total_loja_tipo1['QTD_PRODUTOS_TIPO1_VENDIDOS_DESCONTO'] = total_loja_tipo1['QTD_PRODUTOS_TIPO1_VENDIDOS_DESCONTO'].apply(lambda x: formatar_numero(x, 'numero'))
        total_loja_tipo1['QTD_PRODUTOS_TIPO1'] = total_loja_tipo1['QTD_PRODUTOS_TIPO1'].apply(lambda x: formatar_numero(x, 'numero'))
        total_loja_tipo1['VL_VENDA'] = total_loja_tipo1['VL_VENDA'].apply(lambda x: formatar_numero(x, 'moeda'))



        #INDICADORES DE CAMPANHA DE INCENTIVO
    
        linha = df_campanha['LOJA'] == loja
        coluna = df_campanha.columns.tolist()
        tabela_campanha_loja = df_campanha.loc[linha,coluna].reset_index(drop=True)

        tabela_campanha_loja = tabela_campanha_loja.drop(['LOJA','DESVIO','QTD_META','VENDA', 'VENDA_TOTAL'], axis=1)


        tabela_campanha_loja = tabela_campanha_loja[['CAMPANHA','CLUSTER','PUBLICO','DATA_INICIAL','DATA_FINAL','QTD_TOTAL', 'ALCANCE','VENDA_CAMPANHA','TICKET_MEDIO','RANKING_CLUSTER','PARTCIPACAO_CAMPANHA_LOJA']]


        tabela_campanha_loja = tabela_campanha_loja.rename(columns={'ALCANCE': 'ALCANCE_QTD',
                                                                    'VENDA_CAMPANHA': 'VENDA_CAMP.',
                                                                    'RANKING_CLUSTER': 'RANK_CLUSTER',
                                                                    'PARTCIPACAO_CAMPANHA_LOJA': 'PART._CAMP._LOJA',
                                                                    'DATA_FINAL':'DT_FINAL',
                                                                    'DATA_INICIAL':'DT_INICIAL'
                                                                    })


        assunto = f"PONTO DE PARTIDA - Dia {data_hoje} da V{id_loja}, resultados referente {data_ontem_email}"
        mensagem = f'''<!DOCTYPE html>
        <html lang="en">
        <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Seu E-mail</title>
        <style>
            h3 {{
            text-align: center;
            background-color: yellow;
            color: black;
            }}

            h4 {{
            text-align: left;

            }}



            table {{
            border-collapse: collapse;
            width: 100%;
            border-spacing: 0px;
            text-align: center;
            vertical-align: middle;
            }}

            .center {{
            text-align: center;
            }}

            .yellow-bg {{
            background-color: yellow;
            }}
        </style>
        </head>
        <body>
        <p>Bom Dia,</p>
        
        <p>Os resultados das vendas de ontem <strong>({data_ontem_email}) da {nome_loja}</strong> foi de:</p>

        <h3><b><u>Venda no Dia {data_ontem_email} e Acumulado no Mês</u></b></h3>

        <table border='2'cellpadding="0" cellspacing="0" width="100%" class="center">
            <tr>
            <th>Venda</th>
            <th>Realizado</th>
            <th>Meta</th>
            <th>Desvio</th>
            <th>Alcance</th>
            </tr>

            <tr>
                    <td Align="center">Dia</td>
                    <td Align="center">{formatar_numero(venda_ultimo_dia)}</td>
                    <td Align="center">{formatar_numero(meta_ultimo_dia)}</td>
                    <td Align="center">{formatar_numero(desvio_venda_ultimo_dia)}</td>
                    <td width="100" valign="middle" Align="center">{formatar_numero(alcance_venda_ultimo_dia,"porcentagem")} <font color ="{cor_indicador_dia(alcance_venda_ultimo_dia)}">  ■  </font></td>
                </tr>
            <tr>
                    <td Align="center">Mês</td>
                    <td Align="center">{formatar_numero(venda_acumulado)}</td>
                    <td Align="center">{formatar_numero(meta_acumulada)}</td>
                    <td Align="center">{formatar_numero(desvio_acumulado)}</td>
                    <td width="100" valign="middle" Align="center">{formatar_numero(alcance_acumulado,"porcentagem")} <font color ="{cor_indicador_dia(alcance_acumulado)}">  ■  </font></td>
                </tr> 

            </table> 

            <br></br>
        </table>

        <br>

        <h3><b><u>Meta Sugerida e Ticket Médio do Dia de hoje {data_hoje}</u></b></h3>

        <table class="center">{tabela_meta_email.to_html(index=False)}</table>

        <br>


        <h3><b><u>Venda Não Presencial Acumulado no Mês</u></b></h3>

        <table class="center">{tabela_loja_nao_presencial.to_html(index=False)}</table>

        <br>

        <h3><b><u>NPS Acumulado no Mês</u></b></h3>

        <table> {tabela_nps} </table>

        <br></br>

        <h3><b><u>Venda por COLABORADOR Acumulado no Mês</u></b></h3>

        <table class="center"> {tabela_loja_omni_colaborador.to_html(index=False)} </table>

        <br></br>

        <table class="center"> {total_loja_omni_colaborador.to_html(index=False)} </table>

        <br></br>

        <h3><b><u>FIDELIDADE no Dia {data_ontem_email} por COLABORADOR2</u></b></h3>


        <table class="center"> {base_dia_fidelidade.to_html(index=False)} </table>

        <br></br>

        <table class="center"> {total_loja_fidelidade.to_html(index=False)} </table>

        <br></br>

        <h3><b><u>TMA Acumulado no Mês</u></b></h3>

        <table class="center"> {tabela_loja_tma.to_html(index=False)} </table>


        <br></br>

        <h3><b><u>Desempenho dos COLABORADORES no Mês</u></b></h3>

        <table class="center"> {tabela_loja_desempenho_colaborador.to_html(index=False)} </table>

        <br></br>

        <h3><b><u>TIPO1 no Dia {data_ontem_email}</u></b></h3>
            
        <table class="center"> {tabela_tipo1_dia.to_html(index=False)} </table>

        <br></br>

        <table class="center"> {total_loja_tipo1.to_html(index=False)} </table>

        <br></br>

        <h3><b><u>Campanha de Incentivo no Mês</u></b></h3>
            
        <table class="center"> {tabela_campanha_loja.to_html(index=False)} </table>

            <br></br>

        <h3><b><u>Aniversariante do Dia {data_hoje}  {emoji}</u></b></h3>

        <table class="center"> {tabela_aniversario}</table>

        <p>Att,</p>
        <p>Equipe de B.I</p>
        
        </body>
        </html>
        '''
        enviar_email(lista_email_destinatarios, mensagem, assunto)

        print(f'Email da loja {id_loja} Enviado')
        break

        

        
        